# JRC2018 ROI

This notebook shows how to download neuropile segments (ROIs) for JRC2018U brain template. The ROIs are stored in virtual fly brain, in the dataset with id as JRC2018. 

Templates are stored as nrrd files, with a resolutoin of (1210,566,174)(x:0.52,y:0.52,z:0.1 microns). 

All ROIs are saved in lsa-mcdb-jclowney/lab/Computational_tools/CMTK/templates/JRC2018_ROIs, including nrrd files and a .csv table for their identity.

Reference: [Bogovic et al., 2019](https://v2.virtualflybrain.org/org.geppetto.frontend/org.geppetto.frontend?id=doi_10_1101_376384)

In [6]:
import os 
import pathlib
# connect to virtual fly brain
from vfb_connect import vfb

# set working directory to store ROIs on the server
work_dir=pathlib.Path("/Volumes/lsa-mcdb-jclowney/lab/Computational_tools/CMTK/templates/JRC2018_ROIs")


# set up download function
def download_nrrd_image(vc, vfb_id, output_folder, template="JRC2018U"):
    """Downloads a single NRRD image from VFB for a given ID.

    This function retrieves information for a given Virtual Fly Brain (VFB) ID,
    finds the image URL for a specified template, and downloads it as an
    NRRD file. The output folder will be created if it doesn't exist.

    Args:
        vc: An instance of vfb_connect.cross_server_tools.VfbConnect.
        vfb_id (str): The VFB ID of the image to download (e.g., "VFB_00102745").
        output_folder (str): The path to the folder where the image will be saved.
        template (str, optional): The template space of the image to download.
            Defaults to "JRC2018U".
    """
    import os
    import urllib.request
    import re
    
    try:
        vcinfo = vc.neo_query_wrapper.get_anatomical_individual_TermInfo([vfb_id], summary=False)
        if not vcinfo:
            print(f"No information found for VFB ID: {vfb_id}")
            return
    except Exception as e:
        print(f"Failed to retrieve information for {vfb_id}. Error: {e}")
        return

    term_info = vcinfo[0].get('term', {})
    label = term_info.get('core', {}).get('label', 'unknown_label')
    label = label.split(" ")[0]
    short_form = term_info.get('core', {}).get('short_form', vfb_id)

    # Sanitize label and short_form for use in a filename
    safe_label = re.sub(r'[\\/*?:"<>|]', "", label)
    safe_short_form = re.sub(r'[\\/*?:"<>|]', "", short_form)

    os.makedirs(output_folder, exist_ok=True)

    found_image = False
    for i, ch in enumerate(vcinfo[0].get('channel_image', [])):
        try:
            template_symbol = ch['image']['template_anatomy']['symbol']
            print(f"Channel {i}: Found image with template '{template_symbol}'.")
            if template_symbol == template:
                image_link = ch['image']['image_nrrd']
                
                file_name = f"{safe_label}_{safe_short_form}_{template_symbol}.nrrd"
                file_path = os.path.join(output_folder, file_name)
                
                print(f"Downloading {file_name} to {output_folder}...")
                urllib.request.urlretrieve(image_link, file_path)
                print("Download complete.")
                found_image = True
        except KeyError:
            print(f"Channel {i}: No template information or NRRD link found.")
        except Exception as e:
            print(f"An error occurred while processing channel {i}: {e}")

    if not found_image:
        print(f"No image found with the template '{template}' for VFB ID {vfb_id}.")


In [3]:
# Step 1: download the ROI list 
JRC2018 = vfb.get_instances_by_dataset('JRC2018')
JRC2018.to_csv(work_dir / "JRC2018_ROI_list.csv")
JRC2018.head()

,label,symbol,id,tags,parents_label,parents_id,data_source,accession,xrefs,templates,dataset,license,description
0,SIP on JRC2018Unisex adult brain,,VFB_00102164,"[Entity, Individual, VFB, Adult, Anatomy, Nerv...",[superior intermediate protocerebrum],[FBbt_00045032],[],[],[],[JRC2018Unisex],[JRC2018],[https://creativecommons.org/licenses/by-nc-sa...,NaN
1,ATL on JRC2018Unisex adult brain,,VFB_00102190,"[Entity, Individual, VFB, Adult, Anatomy, Nerv...",[antler],[FBbt_00045039],[],[],[],[JRC2018Unisex],[JRC2018],[https://creativecommons.org/licenses/by-nc-sa...,NaN
2,SLP on JRC2018Unisex adult brain,,VFB_00102162,"[Entity, Individual, VFB, Adult, Anatomy, Nerv...",[superior lateral protocerebrum],[FBbt_00007054],[],[],[],[JRC2018Unisex],[JRC2018],[https://creativecommons.org/licenses/by-nc-sa...,NaN
3,IB on JRC2018Unisex adult brain,,VFB_00102185,"[Entity, Individual, VFB, Adult, Anatomy, Nerv...",[inferior bridge],[FBbt_00040050],[],[],[],[JRC2018Unisex],[JRC2018],[https://creativecommons.org/licenses/by-nc-sa...,NaN
4,LAL on JRC2018Unisex adult brain,,VFB_00102140,"[Entity, Individual, VFB, Adult, Anatomy, Nerv...",[adult lateral accessory lobe],[FBbt_00003681],[],[],[],[JRC2018Unisex],[JRC2018],[https://creativecommons.org/licenses/by-nc-sa...,NaN


In [4]:
# establish connection
from vfb_connect.cross_server_tools import VfbConnect
vc = VfbConnect()

Welcome to the Virtual Fly Brain API
See the documentation at: https://virtualflybrain.org/docs/tutorials/apis/

Establishing connections to https://VirtualFlyBrain.org services...
Session Established!

Session Established!



In [8]:
# download all ROIs
for i, vfb_id in enumerate(JRC2018['id'].tolist()):
    download_nrrd_image(vc, vfb_id, work_dir)

print(f"Dolwnloaded {i} ROIs.")

Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.
Download complete.
Download complete.
Channel 0: Found image with template 'JRC2018U'.
Channel 0: Found image with template 'JRC2018U'.